In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# API Credentials
with open('/Users/Daniel Arechiga/.secret/yelp_api.json') as f:
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'San Francisco, CA'
TERM = 'Pizza'

In [7]:
JSON_FILE = "Data/results_in_progress_SF_pizza.json"
JSON_FILE

'Data/results_in_progress_SF_pizza.json'

In [8]:
def create_json_file(JSON_FILE, delete_if_exists = False):
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == True:
        
        if delete_if_exists == True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")

            os.remove(JSON_FILE)
            
            create_json_file(JSON_FILE, delete_if_exists = False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
            
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        folder = os.path.dirname(JSON_FILE)
        
        if len(folder) > 0:
            
            os.makedirs(folder, exist_ok = True)
            
        with open(JSON_FILE, 'w') as f:
            json.dump([],f)

In [9]:
create_json_file(JSON_FILE, delete_if_exists=True)

with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']

results_per_page = len(results['businesses'])

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_SF_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


125

In [10]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/125 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [11]:
fin_df = pd.read_json(JSON_FILE)
display(fin_df.head(), fin_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,QOpqNPKpClL48TVVrYobAw,pizzetta-211-san-francisco,Pizzetta 211,https://s3-media2.fl.yelpcdn.com/bphoto/1uWcAa...,False,https://www.yelp.com/biz/pizzetta-211-san-fran...,1413,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.783684, 'longitude': -122.482974}",[delivery],$$,"{'address1': '211 23rd Ave', 'address2': '', '...",+14153799880,(415) 379-9880,4810.639764
1,PTFxtXS47ZVRCdZIrEWvGw,golden-boy-pizza-san-francisco,Golden Boy Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/SfosPh...,False,https://www.yelp.com/biz/golden-boy-pizza-san-...,4055,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.7997956, 'longitude': -122.408...",[delivery],$,"{'address1': '542 Green St', 'address2': '', '...",+14159829738,(415) 982-9738,4992.437697
2,AQviX2W0XXJqYtc73KaNxQ,marcellos-pizza-san-francisco,Marcello's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/FlGaQ0...,False,https://www.yelp.com/biz/marcellos-pizza-san-f...,763,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 37.76212241225501, 'longitude': -...","[pickup, delivery]",$,"{'address1': '420 Castro St', 'address2': '', ...",+14158633900,(415) 863-3900,168.444462
3,mSMZJj2pFvttWLpcDmgrEA,tonys-pizza-napoletana-san-francisco,Tony's Pizza Napoletana,https://s3-media3.fl.yelpcdn.com/bphoto/rrBleP...,False,https://www.yelp.com/biz/tonys-pizza-napoletan...,6077,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 37.8003315377662, 'longitude': -1...",[delivery],$$,"{'address1': '1570 Stockton St', 'address2': '...",+14158359888,(415) 835-9888,5002.084941
4,__I9HmtBMV4dDkEgT22V4g,little-star-pizza-divisadero-san-francisco-2,Little Star Pizza - Divisadero,https://s3-media2.fl.yelpcdn.com/bphoto/pSPNqM...,False,https://www.yelp.com/biz/little-star-pizza-div...,3026,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.77753, 'longitude': -122.43798}",[delivery],$$,"{'address1': '846 Divisadero St', 'address2': ...",+14154411118,(415) 441-1118,1855.163788


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,01PEEGsmzViln0LKbQNG2Q,daily-driver-san-francisco-2,Daily Driver,https://s3-media2.fl.yelpcdn.com/bphoto/5w9YPU...,False,https://www.yelp.com/biz/daily-driver-san-fran...,308,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",3.5,"{'latitude': 37.75756, 'longitude': -122.38782}","[delivery, pickup]",NaN,"{'address1': '2535 3rd St', 'address2': '', 'a...",+14158523535,(415) 852-3535,4283.771144
996,_Iupdt0IslH_jU8zkKcWLA,pica-pica-arepa-kitchen-san-francisco,Pica Pica Arepa Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/hAWS9I...,False,https://www.yelp.com/biz/pica-pica-arepa-kitch...,1629,"[{'alias': 'gluten_free', 'title': 'Gluten-Fre...",4.0,"{'latitude': 37.76646996177495, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '401 Valencia St', 'address2': ''...",+14154005453,(415) 400-5453,1420.330189
997,neFTmvR2d49KwFD3Bni8-Q,house-of-sushi-san-francisco,House of Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/38khay...,False,https://www.yelp.com/biz/house-of-sushi-san-fr...,236,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",3.5,"{'latitude': 37.79663, 'longitude': -122.4222}","[delivery, pickup]",$$,"{'address1': '2157 Polk St', 'address2': '', '...",+14154411797,(415) 441-1797,4173.848688
998,rtLxBQILSBEuWE9bxdHRoA,east-end-alameda,East End,https://s3-media4.fl.yelpcdn.com/bphoto/4hUcA8...,False,https://www.yelp.com/biz/east-end-alameda?adju...,545,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 37.7676558, 'longitude': -122.239...",[delivery],$$,"{'address1': '1650 Park St', 'address2': 'Unit...",+15102639630,(510) 263-9630,17296.941991
999,ptlwIBbhC6m6TdcQscf54A,mels-drive-in-san-francisco,Mel's Drive-In,https://s3-media1.fl.yelpcdn.com/bphoto/fzQ4oN...,False,https://www.yelp.com/biz/mels-drive-in-san-fra...,1039,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",3.0,"{'latitude': 37.7811390000165, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '3355 Geary Blvd', 'address2': ''...",+14153872244,(415) 387-2244,2770.377049
